In [1]:
%load_ext autoreload
%autoreload

import dut_func as dut
import serial
import time
import struct
import numpy as np

from misc import *

# dut.drv.serUSB = 'COM19'
drv = dut.drv

ser = serial.Serial('COM6', 9600, timeout=1)
dut.connect(ser)



In [247]:
dut.disconnect()

Serial port disconnected.


In [9]:
dut.dac_set('PLANE_VPP', 0.3)

DAC initialized to a span from -5 V to 5 V


In [245]:
voltages = drv.pic_adc_read() *2
voltages

array([0.        , 0.06930403, 0.07252747, 0.        , 5.13172161])

Set up various DAC channels

In [7]:
dut.dac_set('DAC_VREF_HI_CMP', 1.5)

In [9]:
dut.dac_set('P_VREF_SH', 2.5)

In [11]:
dut.dac_set('P_VAGC_1', 1.2)

In [13]:
dut.dac_set('P_VAGC_0', 0.7)

In [15]:
dut.dac_set('P_TVDD', 2.5)

In [16]:
dut.dac_set('P_TVDD', 1.)

In [17]:
dut.dac_set('P_VREF_TIA', 0.5)

In [18]:
dut.dac_set('DAC_VREF_ARRAY', 0.5)

In [20]:
dut.dac_set('P_AMP_VREF', 2.5)

In [22]:
dut.dac_set('P_AMP_INPUT', 2.5)

________________

Assert Power GOOD

In [2]:
drv.gpio_pin_set(*PIC_PINS['PWR_GOOD'])

Pulse RESET chip with a rising edge

In [3]:
portName, pinPos = PIC_PINS['NRESET_FULL_CHIP']
drv.gpio_pin_reset(portName, pinPos)
time.sleep(0.1) # want to delay 1us
drv.gpio_pin_set(portName, pinPos)

Send a scan chain data

In [15]:
data = bytes([0x10, 0x20, 0x0c, 0x00, 0x20, 0x01, 0x02])
drv.spi_serial_write(2, data)

In [12]:
data = bytes([0x0f, 0b01010101, 0b01010101, 0b01010101, 0b01010101, 0b01010101, 0b01010101])
drv.spi_serial_write(2, data)

In [221]:
data = bytes([0x01,]) * 120
print(data)
drv.spi_serial_write(1, data)

b'\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01'


In [173]:
for _ in range(20):
    data = bytes([0x10, 0x20, 0x0c, 0x00, 0x20, 0x01, 0x02] * 20)
    drv.spi_serial_write(1, data)

In [294]:
bin(0x0c)

'0b1100'

In [20]:
dut.disconnect()

Serial port disconnected.


In [242]:
# NOTE: first power on requires physical switch actions on DUT board as well as PIC commands

# FIRST: SWITCH microntroller VDD power ON (i.e. power up microcontroller)
time.sleep(1) # want to delay 1us
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
dut.dac_init()

DAC initialized to a span from -5 V to 5 V
